In [501]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [559]:
data = pd.read_csv('../lesson2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [560]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [561]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [562]:
data_train.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [563]:
def weighted_random_recommendation(items_weights, n=5, m = 10):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    n : int
        количество товаров, которые рекомендуем
    m : int
        число товаров с большим весом, из которых выбираем товары для рекомендации
    """
    items_weights.sort_values('weight', ascending=False, inplace=True)
    popular = np.array(items_weights.head(m)['item_id'])
    recs = np.random.choice(popular, size=n, replace=False)
    
    
    return recs.tolist()

Сделаем предсказание

In [564]:
%%time
items_weights = data_train.groupby('item_id')['quantity'].sum().reset_index()
items_weights['quantity'] =items_weights['quantity']
items_weights['weight'] = items_weights['quantity'] / items_weights['quantity'].sum()
items_weights.drop(['quantity'], axis = 1, inplace = True)
items_weights
result['weight_rand_rec'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5, m = 10))
result.head(2)

CPU times: user 7.37 s, sys: 324 ms, total: 7.69 s
Wall time: 7.74 s


,user_id,actual,weight_rand_rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6533889, 1426702, 5668996, 6544236, 6534178]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[397896, 5668996, 1404121, 480014, 6534178]"


In [565]:
result3 = result.copy()

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [566]:
result = pd.read_csv('../lesson2/predictions_basic.csv') # закгрузка predict с семианара
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[8358205, 13671693, 691528, 17105399, 8358017]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[6773090, 13213082, 958601, 1032110, 824784]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


В связи с тем, что после сохранения и открытия файла csv изменились типы данных, выполним дополнительные преобразования

In [567]:
result['random_recommendation'] = result['random_recommendation'].map(lambda x: x[1:-1].split(', '))
result['popular_recommendation'] = result['popular_recommendation'].map(lambda x: x[1:-1].split(', '))
result['itemitem'] = result['itemitem'].map(lambda x: x[1:-1].split(', '))
result['cosine'] = result['cosine'].map(lambda x: x[1:-1].split(', '))
result['tfidf'] = result['tfidf'].map(lambda x: x[1:-1].split(', '))
result['own_purchases'] = result['own_purchases'].map(lambda x: x[1:-1].split(', '))

In [568]:
data_test = data_test.astype({'item_id': str})

In [569]:
result2 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result2.columns=['user_id', 'actual']
result['actual'] = result2['actual']

In [570]:
type(result['actual'].iloc[0][0])

str

In [571]:
def precision_at_k_(recommended_list, bought_list, k=5):
    
    #bought_list = np.array(bought_list)
    #recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [572]:
def precision_at_k_column(recommended_list, bought_list, k=5):
    a = []
    for i in range(recommended_list.shape[0]):
        a.append(precision_at_k_(recommended_list.iloc[i], bought_list[i], k=5))
    return a , np.mean(a)    
    #return np.mean(a)

In [574]:
a, b = precision_at_k_column(result['random_recommendation'], result['actual'], k=5)
print('средний precision_at_k для random_recommendation {}'.format(b))
c, d = precision_at_k_column(result['popular_recommendation'], result['actual'], k=5)
print('средний precision_at_k для popular_recommendation {}'.format(d))
x, y = precision_at_k_column(result3['weight_rand_rec'], result3['actual'], k=5)
print('средний precision_at_k для weight_rand_rec {}'.format(y))
e, f = precision_at_k_column(result['itemitem'], result['actual'], k=5)
print('средний precision_at_k для itemitem {}'.format(f))
g, h = precision_at_k_column(result['cosine'], result['actual'], k=5)
print('средний precision_at_k для cosine {}'.format(h))
j, k = precision_at_k_column(result['tfidf'], result['actual'], k=5)
print('средний precision_at_k для tfidf {}'.format(k))
l, m = precision_at_k_column(result['own_purchases'], result['actual'], k=5)
print('средний precision_at_k для own_purchases {}'.format(m))

средний precision_at_k для random_recommendation 0.0007835455435847209
средний precision_at_k для popular_recommendation 0.15523996082272282
средний precision_at_k для weight_rand_rec 0.022624877571008815
средний precision_at_k для itemitem 0.13692458374143
средний precision_at_k для cosine 0.1329089128305583
средний precision_at_k для tfidf 0.13898139079333988
средний precision_at_k для own_purchases 0.17969311132876264


Лучший результат получился для own_purchases

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

In [588]:
data = pd.read_csv('../lesson2/retail_train.csv')

In [589]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()

In [590]:
np.sort(popularity.sales_value)[::-1]

array([467993.62000001,  42645.75      ,  37981.91      , ...,
            0.        ,      0.        ,      0.        ])

In [591]:
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [592]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [593]:
type(result.iloc[0].actual[2])

numpy.int64

In [594]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [595]:
popularity = popularity.reset_index().sort_values(by=['sales_value', 'index'], ascending=False).set_index('index')
popularity = popularity[:5000]

In [596]:
popularity.shape

(5000, 2)

In [597]:
%%time

items = popularity.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

CPU times: user 234 ms, sys: 4.39 ms, total: 238 ms
Wall time: 242 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9655696, 922127, 860776, 956672, 850133]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1050741, 831063, 1067606, 1076105, 930666]"


In [598]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity.item_id.unique()[:5]

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 3.09 ms, sys: 245 µs, total: 3.34 ms
Wall time: 3.26 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9655696, 922127, 860776, 956672, 850133]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1050741, 831063, 1067606, 1076105, 930666]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [599]:
def weighted_random_recommendation(items_weights, n=5, m = 10):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    n : int
        количество товаров, которые рекомендуем
    m : int
        число товаров с большим весом, из которых выбираем товары для рекомендации
    """
    items_weights.sort_values('weight', ascending=False, inplace=True)
    popular = np.array(items_weights.head(m)['item_id'])
    recs = np.random.choice(popular, size=n, replace=False)
    
    
    return recs.tolist()

In [600]:
items_weights = popularity.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weight'] = items_weights['sales_value'] / items_weights['sales_value'].sum()
items_weights.drop(['sales_value'], axis = 1, inplace = True)

In [601]:
result['weight_rand_rec'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5, m = 10))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weight_rand_rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9655696, 922127, 860776, 956672, 850133]","[6534178, 6533889, 1029743, 6534166, 1082185]","[916122, 1029743, 6533889, 1082185, 6534166]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1050741, 831063, 1067606, 1076105, 930666]","[6534178, 6533889, 1029743, 6534166, 1082185]","[995242, 6533765, 6534178, 1082185, 1106523]"


In [602]:
type(result['random_recommendation'][7][0])

int

In [603]:
a, b = precision_at_k_column(result['random_recommendation'], result['actual'], k=5)
print('средний precision_at_k для random_recommendation {}'.format(b))
c, d = precision_at_k_column(result['popular_recommendation'], result['actual'], k=5)
print('средний precision_at_k для popular_recommendation {}'.format(d))
e, f = precision_at_k_column(result['weight_rand_rec'], result['actual'], k=5)
print('средний precision_at_k для weight_rand_rec {}'.format(f))

средний precision_at_k для random_recommendation 0.005484818805093046
средний precision_at_k для popular_recommendation 0.15523996082272282
средний precision_at_k для weight_rand_rec 0.12693437806072477


Результаты на всех товарах:
средний precision_at_k для random_recommendation 0.0007835455435847209
средний precision_at_k для popular_recommendation 0.15523996082272282
средний precision_at_k для weight_rand_rec 0.022624877571008815

Вывод - результаты на бейзлайнах улучшились, когда взяли топ 5000 товаров(кроме popular_recommendation так как рассчет одинаковый)

**Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  𝐾**

In [604]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [605]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [606]:
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [607]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [608]:
np.sort(data.item_id.unique())

array([   25671,    26081,    26093, ..., 18000012, 18024155, 18024556])

In [609]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [611]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 1.54 s, sys: 93 ms, total: 1.63 s
Wall time: 2.21 s


In [612]:
recs

[(2381, 78679.0),
 (3408, 72173.0),
 (2148, 57995.0),
 (3947, 17272.0),
 (3587, 14417.0)]

In [613]:
[id_to_itemid[rec[0]] for rec in recs]

[999999, 1082185, 981760, 1127831, 1098066]

In [614]:
%%time

result['itemitem_k5'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 72 ms, sys: 3.07 ms, total: 75.1 ms
Wall time: 79.2 ms


изменим количество соседей на 10

In [616]:
%%time

model2 = ItemItemRecommender(K=10, num_threads=4) # K - кол-во билжайших соседей
model2.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs2 = model2.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 1.5 s, sys: 26 ms, total: 1.52 s
Wall time: 1.53 s


In [618]:
%%time

result['itemitem_k10'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model2.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 87.3 ms, sys: 1.77 ms, total: 89.1 ms
Wall time: 91.9 ms


изменим количество соседей на 15

In [620]:
%%time
model3 = ItemItemRecommender(K=15, num_threads=4) # K - кол-во билжайших соседей
model3.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs3 = model3.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

result['itemitem_k15'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model3.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])



CPU times: user 1.63 s, sys: 25 ms, total: 1.65 s
Wall time: 1.68 s


In [621]:
result.head(10)

,user_id,actual,random_recommendation,popular_recommendation,weight_rand_rec,itemitem_k5,itemitem_k10,itemitem_k15
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9655696, 922127, 860776, 956672, 850133]","[6534178, 6533889, 1029743, 6534166, 1082185]","[916122, 1029743, 6533889, 1082185, 6534166]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1050741, 831063, 1067606, 1076105, 930666]","[6534178, 6533889, 1029743, 6534166, 1082185]","[995242, 6533765, 6534178, 1082185, 1106523]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1084310, 1097001, 848319, 9211125, 852035]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1029743, 5569230, 6533765, 995242]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 840361, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1113780, 1092502, 1069334, 943393, 1025641]","[6534178, 6533889, 1029743, 6534166, 1082185]","[995242, 5569230, 6533765, 1106523, 6534166]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 1098066]","[999999, 1082185, 981760, 995242, 840361]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[865174, 903738, 907647, 1001277, 862799]","[6534178, 6533889, 1029743, 6534166, 1082185]","[916122, 6534178, 995242, 6534166, 1029743]","[999999, 1082185, 981760, 1127831, 1098066]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 995242, 840361]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[1070272, 13842088, 961980, 897752, 1130517]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 6533889, 6534166, 1106523, 6534178]","[999999, 1082185, 981760, 995242, 1098066]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 1098066, 995242]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[6944571, 995151, 5995597, 1054262, 925626]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534166, 6534178, 995242, 6533889, 5569230]","[999999, 1082185, 981760, 1098066, 1127831]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 995242, 840361]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[5591154, 837270, 919766, 1031316, 8090509]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6533889, 6534166, 916122, 1082185, 1029743]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 995242]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[9296855, 993466, 904129, 1088835, 1118606]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 5569230, 6533889, 6534178, 1029743]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]"
9,16,"[1062973, 1082185, 13007710]","[1102927, 12487154, 1055474, 9707240, 13382073]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6533889, 995242, 6534166, 6533765, 6534178]","[999999, 1082185, 981760, 995242, 1029743]","[999999, 1082185, 981760, 840361, 1098066]","[999999, 1082185, 981760, 995242, 840361]"


In [622]:
e, f = precision_at_k_column(result['itemitem_k5'], result['actual'], k=5)
print('средний precision_at_k для itemitem_k5 {}'.format(f))
g, h = precision_at_k_column(result['itemitem_k10'], result['actual'], k=5)
print('средний precision_at_k для itemitem_k10 {}'.format(h))
j, k = precision_at_k_column(result['itemitem_k15'], result['actual'], k=5)
print('средний precision_at_k для itemitem_k15 {}'.format(k))

средний precision_at_k для itemitem_k5 0.13692458374143
средний precision_at_k для itemitem_k10 0.1509304603330069
средний precision_at_k для itemitem_k15 0.15318315377081293


Вывод, при увеличении количества соседей качество улучшается

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар? Зачем он нужен. Используя этот товар мы смещяем качество рекомендаций. В какую сторону?  Можно ли удалить этот товар? Уберите этот товар и сравните с качеством на семинаре.

Товар 999999 мы используем для того, чтобы обозначить любой другой товар, не вошедший в топ5000. 
Используя этот товар мы ухудшаем качетсво рекомендаций, но значительно уменьшаем время работы алгоритма

In [623]:
data_train.shape

(2278490, 12)

In [655]:
data_train_1 = data_train.loc[data_train['item_id'] != 0]
data_train_1.shape

(1448716, 12)

In [669]:
data_train.loc[data_train['item_id'] == 0, 'item_id'] =6534178

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [670]:
data_train[data_train['item_id'] == 6534178]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
6,2375,26984851516,1,6534178,1,1.57,364,-0.68,1642,1,0.0,0.0
8,2375,26984851516,1,6534178,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6534178,1,2.00,364,-0.79,1642,1,0.0,0.0
10,2375,26984851516,1,6534178,1,2.00,364,-0.79,1642,1,0.0,0.0
12,1364,26984896261,1,6534178,1,2.99,31742,-0.40,1520,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2282311,2088,41297771158,635,6534178,1,15.65,304,-5.25,1258,91,0.0,0.0
2282314,1541,41297771177,635,6534178,1,1.05,304,0.00,1300,91,0.0,0.0
2282315,1541,41297771180,635,6534178,1,5.19,304,0.00,1301,91,0.0,0.0
2282316,1168,41297772063,635,6534178,1,12.40,304,0.00,1526,91,0.0,0.0


In [656]:
#data_train.loc[data_train["item_id"]==999999, "item_id"] = 0

In [671]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [672]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.328651460584234

In [673]:
np.sort(data.item_id.unique())

array([   25671,    26081,    26093, ..., 18000012, 18024155, 18024556])

In [674]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [675]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 1.61 s, sys: 47.8 ms, total: 1.66 s
Wall time: 1.95 s


In [676]:
%%time

result['itemitem_without99_v2'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 89.3 ms, sys: 2.15 ms, total: 91.5 ms
Wall time: 113 ms


In [679]:
result.head(3)

,user_id,actual,random_recommendation,popular_recommendation,weight_rand_rec,itemitem_k5,itemitem_k10,itemitem_k15,itemitem_without99,itemitem_without99_v2
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9655696, 922127, 860776, 956672, 850133]","[6534178, 6533889, 1029743, 6534166, 1082185]","[916122, 1029743, 6533889, 1082185, 6534166]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]","[0, 1082185, 981760, 1127831, 995242]","[6534178, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1050741, 831063, 1067606, 1076105, 930666]","[6534178, 6533889, 1029743, 6534166, 1082185]","[995242, 6533765, 6534178, 1082185, 1106523]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]","[0, 1082185, 981760, 1098066, 995242]","[6534178, 1082185, 981760, 1098066, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1084310, 1097001, 848319, 9211125, 852035]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1029743, 5569230, 6533765, 995242]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 840361, 995242]","[0, 1082185, 981760, 1127831, 995242]","[6534178, 1082185, 981760, 1127831, 995242]"


In [678]:
e, f = precision_at_k_column(result['itemitem_k5'], result['actual'], k=5)
print('средний precision_at_k для itemitem_k5 {}'.format(f))
g, h = precision_at_k_column(result['itemitem_without99'], result['actual'], k=5)
print('средний precision_at_k для itemitem_without99 {}'.format(h))
j, k = precision_at_k_column(result['itemitem_without99_v2'], result['actual'], k=5)
print('средний precision_at_k для itemitem_without99_v2 {}'.format(k))

средний precision_at_k для itemitem_k5 0.13692458374143
средний precision_at_k для itemitem_without99 0.13682664054848187
средний precision_at_k для itemitem_without99_v2 0.17492654260528892


Вот результаты с разными данными
- первый результат как на вебинаре с товаром 999999
- второй результат с товаром 0 вместо 999999. Пыталась сделать следующее - убрать вообще в трейне записи про товары, которые не вошли в топ 5000, но на последнем этапе когда делаются рекомендации появляется ошибка 650. так и не смогла разобраться почему( но видимо связано с размерностью данных, может user_id)
- в третьем случае заменила все товары, которые не вошли в топ 5000 на товар, который вошел в этот рейтинг

Лучший результат получился когда мы используем только топ5000 товаров

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.